# Introduction
 This notebook provides a comprehensive guide for loading, preprocessing, and training a text classification model using the Hugging Face Transformers library. We will use the Roberta tokenizer and a smaller Roberta model to manage computational resources efficiently. The process includes data loading, sampling, tokenization, and model preparation.

# Step 1: Load and Sample Data
 First, we load the dataset from a CSV file and take a random sample to reduce the dataset size for quicker processing.


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Load data
df = pd.read_csv("/kaggle/input/lmsys-chatbot-arena/train.csv")

# Sample 10% of the data
df_sample = df.sample(frac=0.1, random_state=42)

# Check sample data
print(df_sample.head())

# Step 2: Split Data into Training and Validation Sets
We split the data into training and validation sets to evaluate the model's performance.

In [ ]:
# Split data into training and validation sets
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df_sample['prompt'].tolist(), 
    df_sample['winner_model_a'], 
    test_size=0.1, 
    random_state=42
)

# Step 3: Load and Prepare the Tokenizer
We use a smaller Roberta model tokenizer for efficiency and tokenize the text data.

In [ ]:
from transformers import RobertaTokenizer

# Load tokenizer
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

# Tokenize function
def tokenize_function(texts):
    return tokenizer(
        texts,
        padding="max_length",
        truncation=True,
        max_length=128,
        return_tensors='pt'
    )

# Tokenize texts
train_encodings = tokenize_function(train_texts)
val_encodings = tokenize_function(val_texts)

# Check tokenized data
print(train_encodings)
print(val_encodings)

# Step 4: Prepare the Model Configuration
 Set up the configuration for the model, ensuring it uses the appropriate device (GPU if available).

In [ ]:
from transformers import Trainer, TrainingArguments, AutoModelForSequenceClassification
from torch.utils.data import Dataset
import torch
import numpy as np

# Config class
class Config:
    def __init__(self):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.batch_size = 16

cfg = Config()

# Load model
model = AutoModelForSequenceClassification.from_pretrained("roberta-base", num_labels=2).to(cfg.device)

# CustomDataset class
class CustomDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = np.array(labels, dtype=int)
    
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx], dtype=torch.long) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.long)
        return item
    
    def __len__(self):
        return len(self.labels)

# Create datasets
train_dataset = CustomDataset(train_encodings, train_labels)
val_dataset = CustomDataset(val_encodings, val_labels)

# TrainingArguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=cfg.batch_size,
    per_device_eval_batch_size=cfg.batch_size,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

# Train model
trainer.train()

# Step 5: Inference Function
Define a function to perform inference on the tokenized data using the trained model.

In [ ]:
def infer(model, input_ids, attention_mask, batch_size=cfg.batch_size):
    model.eval()
    results = []
    with torch.no_grad():
        for i in range(0, len(input_ids), batch_size):
            batch_input_ids = input_ids[i:i + batch_size].to(cfg.device)
            batch_attention_mask = attention_mask[i:i + batch_size].to(cfg.device)
            outputs = model(input_ids=batch_input_ids, attention_mask=batch_attention_mask)
            results.extend(outputs.logits.cpu().numpy())
    return np.array(results)

# Step 6: Evaluate and Save Results
Tokenize the test data, perform inference, and save the results to a CSV file.

In [ ]:
# Tokenize the test data
test_texts = df_sample['prompt'].tolist()
test_encodings = tokenize_function(test_texts)

# Perform inference
results = infer(model, test_encodings['input_ids'], test_encodings['attention_mask'])

# Convert results to DataFrame and save as CSV
results_df = pd.DataFrame(results, columns=['logit_a', 'logit_b'])
submission = pd.concat([df_sample[['id']], results_df], axis=1)
submission.to_csv('submission.csv', index=False)

# Conclusion
This notebook provided a structured approach to loading, preprocessing, and training a text classification model using the Transformers library. The steps included data sampling, tokenization, model configuration, and inference. The final results were saved for further analysis. This methodology ensures efficient use of computational resources while maintaining model performance.